### Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

1.
Задание

Загрузите датасет. Описание датасета можно посмотреть [здесь](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/).

In [1]:
import pandas as pd

2.
Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [5]:
data = pd.read_csv('SMSSpamCollection.txt', sep ='\t', names = ['target', 'text'])
data.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


3.
Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [34]:
y = list(map((lambda x : 1 if x == 'spam' else 0), data.target))
text = list(data.text.values)

4. 
Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

In [87]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)

5.
Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [85]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import random

In [88]:
random.seed(2)
model = LogisticRegression()
print(cross_val_score(model, X, y, cv = 10, scoring = 'f1').mean())

0.9311542822856882


In [45]:
with open("answer1.txt", "w") as f:
    f.write(str(round(0.9311542822856882, 1)))

* пробовал ответы от 0.7 до 1.0 - ни оди не принят грейдером (((

6. 
А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

* "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

* "FreeMsg: Txt: claim your reward of 3 hours talk time"

* "Have you visited the last lecture on physics?"

* "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

* "Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [46]:
model.fit(X, y)

LogisticRegression()

In [52]:
examples = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
            "FreeMsg: Txt: claim your reward of 3 hours talk time",
            "Have you visited the last lecture on physics?",
            "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
            "Only 99$"]
X_test = vectorizer.transform(examples)

In [63]:
res = model.predict(X_test)
res

array([1, 1, 0, 0, 0])

In [55]:
with open("answer2.txt", "w") as f:
    f.write('1 1 0 0 0')

7.
Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. 

В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [57]:
from sklearn.pipeline import Pipeline

In [67]:
model = LogisticRegression()
X_train = []
vectorizer1 = CountVectorizer(ngram_range=(2,2))
X_train.append(vectorizer1.fit_transform(text))
vectorizer2 = CountVectorizer(ngram_range=(3,3))
X_train.append(vectorizer2.fit_transform(text))
vectorizer3 = CountVectorizer(ngram_range=(1,3))
X_train.append(vectorizer3.fit_transform(text))

In [68]:
y_pred = []
for i in range(3):
    y_pred.append(cross_val_score(model, X_train[i], y, cv = 10, scoring = 'f1').mean())

y_pred

[0.8168951028736983, 0.7249691484935827, 0.9216545237836128]

In [76]:
with open("answer3.txt", "w") as f:
    f.write(' '.join(str(round(y_pred[i], 2)) for i in range(len(y_pred))))

8.
Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

 По какой-то причине  обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно  посчитайте частоты слов и обучите классификатор.  

In [77]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

In [78]:
y_pred = []
for i in range(3):
    y_pred.append(cross_val_score(model, X_train[i], y, cv = 10, scoring = 'f1').mean())

y_pred

[0.6457761003991855, 0.37861105465217015, 0.888454144419353]

In [79]:
with open("answer4.txt", "w") as f:
    f.write(' '.join(str(round(y_pred[i], 2)) for i in range(len(y_pred))))

9.
Попробуйте использовать в логистической регрессии в качестве признаков $Tf*idf$ из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к $tf*idf$ не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [81]:
vectorizerTF = TfidfVectorizer(ngram_range=(1,3))
X = vectorizer1.fit_transform(text)

In [82]:
print(cross_val_score(model, X, y, cv = 10, scoring = 'f1').mean())

0.6457761003991855


In [83]:
with open("answer5.txt", "w") as f:
    f.write('-1')